In [176]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 0.5

brand = []
product_type = []
name = []
product_url = []
timestamp = []
picture_url = []
price = []
sales_price = []

#URL = 'https://shop.diesel.com/en/womens/apparel/'
URL = 'https://shop.diesel.com/en/mens/apparel/'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

last_height = driver.execute_script("return document.body.scrollHeight")
for i in range (37, 771, 36): # change variable see how many pages

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div/div[1]/div/div[2]/div[{}]/div/button'.format(i)).click()

soup = BeautifulSoup(driver.page_source,'html.parser')

for i in soup.find_all('div',{'class':'row-span-1 col-span-1 js_tile'}):
    brand.append('Diesel')
    product_type.append("Men's Clothing")  # change variable
    timestamp.append(date_today)
    
    name.append(i.find('div',{'itemprop':'name'}).a.text)
    product_url.append("https://shop.diesel.com" + i.find('div',{'itemprop':'name'}).a.get('href'))
    if i.find('div',{'class':"image-container"}).img.get('data-src') != None:
        picture_url.append(i.find('div',{'class':"image-container"}).img.get('data-src'))
    else:
        picture_url.append(i.find('div',{'class':"image-container"}).img.get('src'))
    if i.find('div',{'class':'price'}).find('span',{'class':'strike-through list'}) != None:
        price.append(i.find('div',{'class':'price'}).find('span',{'class':'strike-through list'}).text.strip())
    else:
        price.append(None)
    sales_price.append(i.find('div',{'class':'price'}).span.find('span',{'class':'sales'}).text.strip())

# Second Layer

In [177]:
color = []
product_description = []
material = []
picture_name = []
counter = 1

for link in product_url:
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    color_item = []
    if soup.find('span',{'class':'attribute-label-value single-val selected'}) != None:
        color_item.append(soup.find('span',{'class':'attribute-label-value single-val selected'}).text.strip())
    else:
        for i_color in soup.find('ul',{'class':'product-attributes js_product-attributes'}).li.find_all('li'):
            color_item.append(i_color.get('data-attr-display-value'))
    color.append(color_item)
    product_description.append(soup.find('div',{'class':'product-description'}).text.strip())
    material_i = []
    for mat_i in soup.find('div',{'class':'product-details'}).find_all('div',{'class':'product-details-element'}):
        material_i.append(mat_i.text.strip().replace('\n',''))
    material.append(material_i)
    picture_name.append('diesel_men_' + str(counter) + '.jpg') # change variable
    counter += 1

In [181]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp

# Remove Duplicates

In [179]:
result_df = second_pass.drop_duplicates(subset=['product_url'], keep='first')
result_df.to_csv("diesel-men.csv", index=False, encoding = 'utf-8-sig')

# Download Image

In [180]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/diesel-men.csv', encoding='utf-8-sig')

os.mkdir('diesel-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('diesel-men/'+ "diesel-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1